### Import Libraries

In [1]:
import re
import numpy as np
import pandas as pd
import datetime as dt

### Read Data

In [2]:
# Read raw data
df = pd.read_csv("property_listings.csv")

# Replace all unknown with NaN
df = df.replace('unknown',np.nan)

df.head(5)

,address,address_2,nearest_mrt,time_to_nearest_mrt,price,unit_type,room_type,room_size,room_size_2,status,updated_time,link
0,Common Room in 111 Pasir Ris Street 11,NaN,NaN,NaN,"$1,000/mo",HDB,Common room,150 sqft / 13.94 sqm,NaN,FULLY FURNISHED,Updated 2 hrs ago,/singapore/rooms/property/111-pasir-ris-street...
1,Common Room in Sun Plaza,30 Sembawang Dr · Condo,Sembawang,1 mins,"$1,400/mo",NaN,Common room,150 sqft / 13.94 sqm,NaN,FULLY FURNISHED,Updated 4 hrs ago,/singapore/rooms/property/sun-plaza-condo-U6bm...
2,Common Room in East Mews,853 Mountbatten Rd · Condo,NaN,NaN,"$1,300/mo",NaN,Common room,100 sqft / 9.29 sqm,NaN,FULLY FURNISHED,Updated 5 hrs ago,/singapore/rooms/property/east-mews-condo-9VAr...
3,Room in Lotus at Paya Lebar (East Wing),11 Paya Lebar Rd · Condo,Paya Lebar,4 mins,"$3,480/mo",NaN,1 Bath,660 sqft / 61.32 sqm,NaN,NaN,Updated 5 hrs ago,/singapore/rooms/property/lotus-at-paya-lebar-...
4,Room in Eunos Crescent View,NaN,Eunos,7 mins,"$2,600/mo",Conservation House,400 sqft / 37.16 sqm,NaN,NaN,FULLY FURNISHED,Updated 5 hrs ago,/singapore/rooms/property/property-in-singapor...


### Cleaning Address Column

In [3]:
# Remove room type in front of address
df['address'] = df['address'].str.replace('Common Room in|Master Room in|Room in', '', regex=True)

# Assign 'Condo' to the "unit_type" column where "address_2" contains '.Condo'
df.loc[df['address_2'].str.contains('· Condo', case=False, na=False), 'unit_type'] = 'Condo'

# Remove '.Condo' from the "address_2" column
df['address_2'] = df['address_2'].str.replace('· Condo', '', regex=False)

# Combine condo info from 'address_2' and 'address' columns
df['address'] = df.apply(lambda row: row['address_2'] + ', ' + row['address'] if isinstance(row['address_2'], str) else row['address'], axis=1)

# Remove everything after the comma, but only if there's something in front of the comma
df['address'] = df['address'].apply(lambda x: x.replace('  ,  ', '') if x.startswith('  ,') else x)
df['address'] = df['address'].str.replace(r' ,.*', '', regex=True)

# Remove any leading and trailing spaces
df['address'] = df['address'].str.strip()

# Drop address_2 column
df = df.drop('address_2', axis=1)

### Cleaning Price Column

In [4]:
# Remove the '$' and ',' characters
df['price'] = df['price'].str.replace('[\$,]', '', regex=True)

# Remove the "/mo" part
df['price'] = df['price'].str.replace('/mo', '', regex=True)

# Convert to float
df['price'] = df['price'].astype(float)

# Filter out extreme value
df = df[df['price'] <= 50000]

### Cleaning Unit Type Column

In [5]:
# Define a dictionary to map other HDB types to 'HDB'
hdb_mapping = {
    'HDB (EM)': 'HDB',
    'HDB (3I)': 'HDB',
    'HDB (EA)': 'HDB',
    'HDB (3A)': 'HDB',
    'HDB (4A)': 'HDB',
    'HDB (5I)': 'HDB',
    'HDB (3NG)': 'HDB',
    'HDB (5A)': 'HDB',
    'HDB (3STD)': 'HDB',
    'HDB (4NG)': 'HDB',
    'HDB (4I)': 'HDB',
    'HDB (4S)': 'HDB',
    'HDB (5RM)': 'HDB',
    'HDB (3S)': 'HDB',
    'HDB (1RM)': 'HDB',
    'HDB (4STD)': 'HDB',
    'HDB (3RM)': 'HDB',
    'HDB (4RM)': 'HDB',
    'HDB (5STD)': 'HDB',
    'HDB (2A)': 'HDB',
    'HDB (Multi-gen)': 'HDB',
    'HDB (2STD)': 'HDB'
}

# Use the replace method to map other HDB types to 'HDB'
df['unit_type'] = df['unit_type'].replace(hdb_mapping)

# Fill the missing values with Unknown as the data is still valuable for analysis of other fields
df['unit_type'].fillna('Unknown', inplace=True)

### Cleaning Room Size Column

In [6]:
# Apply the function to gather all the room size data that spread across 3 columns
def extract_room_size(row):
    if isinstance(row['room_size'], str) and 'sqft' in row['room_size']:
        return row['room_size']
    elif isinstance(row['room_size_2'], str) and 'sqft' in row['room_size_2']:
        return row['room_size_2']
    elif isinstance(row['room_type'], str) and 'sqft' in row['room_type']:
        return row['room_type']
    else:
        return np.nan

df['room_size'] = df.apply(extract_room_size, axis=1)

# Drop the other unused column
df = df.drop('room_size_2', axis=1)

# Remove unnecessary symbol and change the unit of room size to float
df['room_size'] = df['room_size'].str.split(' / ').str[0].str.replace(',', '').str.replace(' sqft', '').astype(float)

# Filter out extreme value
df = df[df['room_size'] <= 50000]

# Replace room sizes smaller than 50 with 50
df['room_size'] = df['room_size'].apply(lambda x: 50 if x < 50 else x)

''' 
Problem Encountered:
The room size data extracted from the website is not entirely accurate. 
Some room sizes are recorded as 0, while others do not align with the associated prices. 
For instance, you may pay 3k just to rent a 150 sqft room.
'''

### Cleaning Room Type Column

In [7]:
# Function to remove the keyword from the room_type column
def extract_room_type(room_type_str):
    keywords = [' sqft', ' Bath']  # Keywords to split on
    for keyword in keywords:
        room_type_str = room_type_str.split(keyword, 1)[0]
    return room_type_str.strip()

# Apply the function to the 'room_type' column
df['room_type'] = df['room_type'].apply(extract_room_type)

# Remove the numeric value and change it to NaN
df['room_type'] = df['room_type'].str.replace(r'\d', '', regex=True)
df['room_type'] = df['room_type'].replace(['',','], np.nan, regex=True)

# Define a function to impute missing room_type based on unit_type and room_size
def impute_room_type(row):
    if pd.isna(row['room_type']):
        unit_type = row['unit_type']
        room_size = row['room_size']

        # Create a dictionary that maps unit_type to room type categorization criteria
        unit_type_criteria = {
            'HDB': {'Common room': 150, 'Master room': 450, 'Whole unit': float('inf')},
            'Condo': {'Common room': 150, 'Master room': 350, 'Studio': 750, 'Whole unit': float('inf')},
            'Terraced House': {'Common room': 150, 'Master room': 350, 'Whole unit': float('inf')},
            'Corner Terrace': {'Common room': 150, 'Master room': 350, 'Whole unit': float('inf')},
            'Shophouse': {'Common room': 150, 'Master room': 350, 'Whole unit': float('inf')},
            'Semi-D': {'Common room': 150, 'Master room': 350, 'Whole unit': float('inf')},
            'Bungalow': {'Common room': 150, 'Master room': 350, 'Whole unit': float('inf')},
            'Cluster House': {'Common room': 150, 'Master room': 350, 'Whole unit': float('inf')},
            'Conservation House': {'Common room': 150, 'Master room': 350, 'Whole unit': float('inf')},
            'Land Only': {'Common room': 150, 'Master room': 350, 'Whole unit': float('inf')},
            'Townhouse': {'Common room': 150, 'Master room': 350, 'Whole unit': float('inf')},
            'Unknown': {'Unknown': float('inf')}
        }

        # Determine the room type based on the unit_type and room_size
        for room_type, criteria in unit_type_criteria[unit_type].items():
            if room_size <= criteria:
                return room_type

    return row['room_type']

# Apply the imputation function to fill missing room_type values
df['room_type'] = df.apply(impute_room_type, axis=1)

# Set room type for rows where both room size and price exceed thresholds
df.loc[(df['room_size'] > 1000) & (df['price'] > 3000), 'room_type'] = 'Whole unit'

### Cleaning Status Column

In [8]:
# Remove unnecessary elements
df['status'] = df['status'].replace(['DIVERSITY FRIENDLY', 'REMOTE VIEWING'], np.nan)

# Impute missing values in the "status" column with the mode
mode_status = df['status'].mode()[0]
df['status'].fillna(mode_status, inplace=True)

### Cleaning Updated Time Column

In [9]:
# Apply the parsing function to the "updated_time" column to obtain the date time
def parse_updated_time(updated_time_str):
    minutes = 0
    hours = 0
    days = 0
    
    # Extract numerical values from the string
    values = re.findall(r'\d+', updated_time_str)
    
    if 'min' in updated_time_str:
        minutes = int(values[0])
    elif 'hour' in updated_time_str:
        hours = int(values[0])
    elif 'day' in updated_time_str:
        days = int(values[0])
    
    return dt.datetime.now() - dt.timedelta(days=days, hours=hours, minutes=minutes)

df['updated_time'] = df['updated_time'].apply(parse_updated_time)

# Extract date from the datetime
df['updated_date'] = df['updated_time'].dt.date

# Change the date format
df['updated_date'] = pd.to_datetime(df['updated_date'], format='%Y-%m-%d')
df['updated_date'] = df['updated_date'].astype(str)


### Cleaning Link Column

In [10]:
# Add "https://99.co" to the 'link' column
df['link'] = 'https://99.co' + df['link']

In [11]:
df.to_csv('Rental_data.csv', index=False)